In [1]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [13]:
row_numbers = np.arange(5,16,1)
print(row_numbers)

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [22]:
res = pd.DataFrame(columns=('column_selected','precision','recall','finding_rate','cross_val'))

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path,header=None, names = ['column_selected','precision','recall','finding_rate','cross_val'] )

filePath = "rows_selected/"
for row_number in row_numbers:
    selected_row_number = "Rows_selected_"+str(row_number)+"_result.csv"
    result = load_one_version_method_matrix(filePath+selected_row_number)
    sorted_result = result.sort_values(by="finding_rate" , ascending=False) 
    res = res.append(sorted_result[0:5], ignore_index=True)


rows_selected/Rows_selected_5_result.csv
rows_selected/Rows_selected_6_result.csv
rows_selected/Rows_selected_7_result.csv
rows_selected/Rows_selected_8_result.csv
rows_selected/Rows_selected_9_result.csv
rows_selected/Rows_selected_10_result.csv
rows_selected/Rows_selected_11_result.csv
rows_selected/Rows_selected_12_result.csv
rows_selected/Rows_selected_13_result.csv
rows_selected/Rows_selected_14_result.csv
rows_selected/Rows_selected_15_result.csv


NameError: name 'np' is not defined

# same in the Classification

In [1]:

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected
def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n
def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def classification_binary(column_selected):

    filePath = "AlltheMatrix/"
    single_version_file_path = filePath + 'modified_single_version-ck-oo.csv'
    changed_file_path = filePath + 'modified_change_metrics.csv'
    bug_matrix_path = filePath + 'modified_bug-metrics.csv' 
    # complexity_code_change_path = 



    filePathNowusing = changed_file_path



    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')

    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == 'classname'):
            startindex = i
        elif (column == 'bugs'):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix['bugs'].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(997)
    target, data = target[shuffle_index], data[shuffle_index]

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(997)
    target, data = target[shuffle_index], data[shuffle_index]

    #split the train set and the test set
    X_train = data[:700]
    X_test = data[700:]
    Y_train = target[:700]
    Y_test = target[700:]
    print(X_train,Y_train.shape)
    #train a binary classifier
    Y_train_bugs = (Y_train > 0)
    Y_test_bugs = (Y_test > 0 )
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    #hardCopy to test the selected rows
    columncombation_j = column_selected#(0, 6, 8, 10, 13, 14)
    X_train_selected = get_selected_rows(columncombation_j,X_train)
    X_test_selected = get_selected_rows(columncombation_j,X_test)


    forest_clf = RandomForestClassifier(n_estimators = 10)
    forest_clf.fit(X_train_selected,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train_selected, Y_train_bugs, cv = 10, scoring = 'accuracy')
    Y_forest_predict = forest_clf.predict(X_train_selected)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')

    print(columncombation_j)
    #    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test_selected)

    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')

In [5]:
import pandas as pd
import os
import numpy as np
classification_binary((0, 6, 8, 10, 13, 14))

AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[39 6 0 ... -0.25640999999999997 367.0 61.6485]
 [36 13 0 ... 3.9722199999999996 333.14300000000003 72.3289]
 [51 7 0 ... 1.03922 367.0 123.074]
 ...
 [16 5 0 ... 1.0625 325.143 102.57600000000001]
 [19 3 0 ... 1.31579 367.0 118.041]
 [239 59 0 ... 4.59414 367.0 80.6008]] (700,)
Forest_accurancy is 0.9844961240310077
Forest_recall is 0.9202898550724637
(0, 6, 8, 10, 13, 14)
fingdingrate = (0.4411764705882353, 30)
